Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount("./content/drive/MyDrive/TECHNOPRO_food_package/Dataset")

MessageError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

pip で必要なライブラリをダウンロードする

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00


Import

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms, datasets, models
from torch.cuda.amp import GradScaler

import pandas as pd
from sklearn.model_selection import KFold
from PIL import Image
import timm

from tqdm import tqdm
import time
import copy
import datetime
import os
import glob

カレントディレクトリを変更

In [ ]:
os.chdir("./drive/MyDrive/TECHNOPRO_food_package/codes")

パラメータ関係

In [ ]:
DATA_DIR_PATH = "../data"
TEST_DIR_PATH = "../Dataset/test/*"
EPOCH = 10
RANDOM_STATE = 2023
CROSS_VALIDATION = False
t_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(t_delta, 'JST')
dt_now = datetime.datetime.now(JST).strftime('%Y%m%d')
if not os.path.isdir(f"../output/{dt_now}"):
    os.mkdir(f"../output/{dt_now}")
TRAIN_RATIO = 0.8
BATCH_SIZE = 16
CLASSES = 2
LEARNING_RATE = 1e-5
WEIGHT_DECAY = 1e-5
MOMENTUM = 0.01

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomVerticalFlip(),
        # transforms.RandomEqualize(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

学習関係

In [ ]:
class Train():
    def __init__(self, model, optimizer, criterion, dataloaders: dict, data_size: dict,
                 epoch: int, learning_rate: float, scaler, scheduler):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.dataloaders = dataloaders
        self.data_size = data_size
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.scaler = scaler
        self.scheduler = scheduler

    def do_train(self):
        if torch.cuda.is_available():
            self.model = self.model.cuda()
            self.criterion = self.criterion.cuda()

        # 途中経過保存用に、リストを持った辞書を作ります。
        loss_dict = {"train": [], "val": []}
        acc_dict = {"train": [], "val": []}

        best_acc = 0.0
        epoch_loss = 0
        epoch_acc = 0

        for epoch in tqdm(range(self.epoch)):
            if (epoch + 1) % 5 == 0:  # 5回に1回エポックを表示します。
                print('Epoch {}/{}'.format(epoch, self.epoch - 1))
                print('-' * 10)

            for phase in ['train', 'val']:
                if phase == 'train':
                    self.model.train()  # 学習モード。dropoutなどを行う。
                else:
                    self.model.eval()  # 推論モード。dropoutなどを行わない。

                running_loss = 0.0
                running_corrects = 0

                for data in self.dataloaders[phase]:
                    inputs, labels = data

                    if torch.cuda.is_available():
                        inputs = inputs.cuda()
                        labels = labels.cuda()

                    outputs = self.model(inputs)
                    _, preds = torch.max(outputs.data, 1)
                    loss = self.criterion(outputs, labels)

                    if phase == 'train':
                        self.optimizer.zero_grad()
                        self.scaler.scale(loss).backward()
                        self.scaler.step(optimizer)
                        self.scaler.update()

                        self.optimizer.step()
                        # self.scheduler.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels)

                    # リストに途中経過を格納
                    loss_dict[phase].append(epoch_loss)
                    acc_dict[phase].append(epoch_acc)

                epoch_loss = running_loss / self.data_size[phase]
                epoch_acc = running_corrects.item() / self.data_size[phase]

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                # print( self.scheduler.get_lr()[0])

                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(self.model.state_dict())
        print('Best val acc: {:.4f}'.format(best_acc))
        return best_model_wts

modelの最終層を変更

In [ ]:
class MyModel(nn.Module):
    def __init__(self, model_name: str, pretrained: bool, hidden_dim: int, out_dim: int):
        super().__init__()
        self.backbone = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=0)
        self.in_features = self.backbone.num_features
        self.head = nn.Sequential(
                nn.Linear(in_features=self.in_features, out_features=hidden_dim),
                nn.ReLU(),
                nn.Linear(in_features=hidden_dim, out_features=out_dim),
                nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        h = self.backbone(x)
        y = self.head(h)

        return y

pytorch models

In [ ]:
def create_model(model_name):
    if model_name == "wide_res":
        model = models.wide_resnet101_2(pretrained=True)
        classifier = nn.Sequential(
            nn.Linear(in_features=1000, out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(in_features=1024, out_features=CLASSES),
            nn.LogSoftmax(dim=1)
        )
        model.add_fc = classifier

    elif model_name == "vit_l_16":
        model = models.vit_l_16(pretrained=True)

        classifier = nn.Sequential(
            nn.Linear(in_features=1000, out_features=100),
            nn.ReLU(),
            nn.Linear(in_features=100, out_features=CLASSES),
            nn.LogSoftmax(dim=1)
        )
        model.fc = classifier

    elif model_name == "vit_l_32":
            model = models.vit_l_16(weights=models.vision_transformer.ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1)
            model.heads[0] = nn.Linear(in_features=model.heads[0].in_features, out_features=CLASSES, bias=True)

    elif model_name == "swin_v2_b":
        model = models.swin_v2_b(weights=models.Swin_V2_B_Weights.IMAGENET1K_V1)
        model.head = nn.Linear(in_features=1024, out_features=CLASSES, bias=True)

    elif model_name == "swin_base_patch4_window7_224":
        model = timm.create_model(model_name=model_name, pretrained=True, num_classes=CLASSES)
    return model

アンサンブルモデル

In [ ]:
class MyEnsembleModel(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD, modelE, input):
        super(MyEnsembleModel, self).__init__()

        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.modelD = modelD
        self.modelE = modelE

        self.add_fc = nn.Linear(input, 2)

    def forward(self, x):
        outA = self.modelA(x)
        outB = self.modelB(x)
        outC = self.modelC(x)
        outD = self.modelD(x)
        outE = self.modelE(x)

        out = outA + outB + outC + outD + outE

        return out

予測関係

In [ ]:
class Pred():
    def __init__(self, model, data_transforms: dict, test_dir_path: str):
        if torch.cuda.is_available():
            self.model = model.cuda()
        else:
            self.model = model

        self.data_transforms = data_transforms
        self.test_dir_path = test_dir_path

    def do_prediction(self) -> pd.DataFrame:
        """学習して後にそのまま予測

        :return: pd.DataFrame
        """
        self.model.eval()
        file_names = []
        predictions = []

        for image_file_path in tqdm(glob.glob(self.test_dir_path)):
            file_name = os.path.basename(image_file_path)
            file_names.append(file_name)
            image = Image.open(image_file_path)
            image = self.data_transforms["val"](image)
            image = image.unsqueeze(0)
            if torch.cuda.is_available():
                image = image.cuda()

            with torch.no_grad():
                output = self.model(image)
                prediction = nn.functional.softmax(output, dim=1)
                prediction = prediction.data[0, 1].item()
                # print(prediction)
                predictions.append(prediction)

        d = {'0': file_names, '1': predictions}
        dst_df = pd.DataFrame(data=d)

        return dst_df

## 学習only

In [ ]:
model_names = ["swin_base_patch4_window7_224", "swin_large_patch4_window7_224", "swin_s3_base_224"]

folds = KFold(shuffle=True, n_splits=5, random_state=RANDOM_STATE)
data = datasets.ImageFolder(root=DATA_DIR_PATH, transform=data_transforms["train"])
train_size = int(TRAIN_RATIO * len(data))
val_size = len(data) - train_size
data_size = {"train": train_size, "val": val_size}
model_name = model_names[0]
print(model_name)
scaler = GradScaler(enabled=True)

for i_fold, (train_idx, val_idx) in enumerate(folds.split(data)): # クロスバリデーション
    print(f"Kfold: {i_fold}")
    # model = create_model(model_name)
    model = MyModel(model_name=model_name, pretrained=True, hidden_dim=100, out_dim=CLASSES)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = lr_scheduler.CyclicLR(optimizer, base_lr=LEARNING_RATE,
                                      max_lr=0.1, step_size_up=10,
                                      step_size_down=10, mode='triangular2',
                                      cycle_momentum=False)
    criterion = nn.CrossEntropyLoss()

    data_train = torch.utils.data.Subset(data, train_idx)
    data_val = torch.utils.data.Subset(data, val_idx)

    train_loader = torch.utils.data.DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = torch.utils.data.DataLoader(data_val, batch_size=BATCH_SIZE, shuffle=False)
    dataloaders = {"train": train_loader, "val": val_loader}

    MODEL_WEIGHT_SAVE_PATH = f"../Models/{dt_now}_{model_name}_epoch{EPOCH}_{i_fold}.pth"

    trainer = Train(model=model, optimizer=optimizer, criterion=criterion,
                    dataloaders=dataloaders, data_size=data_size, epoch=EPOCH,
                    learning_rate=LEARNING_RATE, scaler=scaler,
                    scheduler=scheduler)
    best_model_wts = trainer.do_train()
    del trainer, model, optimizer, criterion
    torch.save(best_model_wts, MODEL_WEIGHT_SAVE_PATH)

    if CROSS_VALIDATION:
        continue
    else:
        break

swin_base_patch4_window7_224
Kfold: 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  0%|          | 0/40 [00:00<?, ?it/s]

train Loss: 0.5305 Acc: 0.7517


  2%|▎         | 1/40 [06:28<4:12:13, 388.03s/it]

...

val Loss: 0.5638 Acc: 0.8761
train Loss: 0.0173 Acc: 0.9943


 98%|█████████▊| 39/40 [1:09:06<01:38, 98.63s/it]

val Loss: 0.6011 Acc: 0.8761
Epoch 39/39
----------
train Loss: 0.0018 Acc: 1.0000


100%|██████████| 40/40 [1:10:44<00:00, 106.12s/it]

val Loss: 0.6564 Acc: 0.8693
Best val acc: 0.9106


## 予測only

In [ ]:
weight_path = f"../Models/{dt_now}_{model_name}_epoch{EPOCH}_0.pth"
PREDICTION_FILE_SAVE_PATH = f"../output/{dt_now}/{model_name}_epoch{EPOCH}.csv"

model = MyModel(model_name=model_name, pretrained=True, hidden_dim=100, out_dim=CLASSES)
# model = create_model(model_name)
model.load_state_dict(torch.load(weight_path))
predictor = Pred(model=model, data_transforms=data_transforms, test_dir_path=TEST_DIR_PATH)
prediction_df = predictor.do_prediction()
prediction_df.to_csv(PREDICTION_FILE_SAVE_PATH, index=False, header=False)

100%|██████████| 2188/2188 [02:12<00:00, 16.48it/s]


アンサンブルで予測(同じモデル)

In [ ]:
model_weight_paths = glob.glob(f"../Models/swin_base_patch4_window7_224/*")

model_0 = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
model_0.load_state_dict(torch.load(model_weight_paths[0]))
model_1 = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
model_1.load_state_dict(torch.load(model_weight_paths[1]))
model_2 = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
model_2.load_state_dict(torch.load(model_weight_paths[2]))
model_3 = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
model_3.load_state_dict(torch.load(model_weight_paths[3]))
model_4 = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
model_4.load_state_dict(torch.load(model_weight_paths[4]))

アンサンブルで予測(異なるモデル)

In [ ]:
swin_base_weight_path = "../Models/swim/20230913_swin_base_patch4_window7_224_epoch20_0.pth"
swin_large_weight_path = "../Models/swim/20230913_swin_large_patch4_window7_224_epoch15_0.pth"
swin_s3_base_weight_path = "../Models/swim/20230913_swin_s3_base_224_epoch15_0.pth"

swin_base_model = MyModel(model_name=model_names[0], pretrained=True, hidden_dim=100, out_dim=CLASSES)
swin_base_model.load_state_dict(torch.load(swin_base_weight_path))
swin_large_model = MyModel(model_name=model_names[1], pretrained=True, hidden_dim=100, out_dim=CLASSES)
swin_large_model.load_state_dict(torch.load(swin_large_weight_path))
swin_s3_base_model = MyModel(model_name=model_names[2], pretrained=True, hidden_dim=100, out_dim=CLASSES)
swin_s3_base_model.load_state_dict(torch.load(swin_s3_base_weight_path))

In [ ]:
ensemble_model = MyEnsembleModel(modelA=model_0, modelB=model_1, modelC=model_2, modelD=model_3, modelE=model_4, input=2)
PREDICTION_FILE_SAVE_PATH = f"../output/{dt_now}/swin_base_patch4_window7_224_ensemble_model.csv"
predictor = Pred(model=ensemble_model, data_transforms=data_transforms, test_dir_path=TEST_DIR_PATH)
prediction_df = predictor.do_prediction()
prediction_df.to_csv(PREDICTION_FILE_SAVE_PATH, index=False, header=False)